## Purpose of this notebook:
- Provide a model that can predict the home value of single unit properties that the tax district assessed during the  months of May through August 2017, using number of bedrooms, number of bathrooms, and square feet of the property as features
- Create and evaluate different models to find the effectiveness of the selected features
- Using the FIPS numbers, find the specific county for each home in the data, and create a seperate dataframe to display the tax rate for each county

### Some Context:
What are the key factors that impact the value of your home?
According to Marian White from [moving.com](https://www.moving.com/tips/7-key-factors-that-impact-the-value-of-your-home/), the top two factors are location and square footage.
>"Ask any Realtor what the most important factor is when it comes to a 
>home’s value, and they’ll tell you: “location, location, location.” 
>The size of the home is the next important factor in determining 
>the value of your home."

## Inital Questions and Hypotheses
### Does square footage affect tax value?
`Null Hypothesis: The average price of homes with 3000 sq.ft. or more is equal to the average price of homes with 3000 sq.ft. or less`

`Alternate Hypothesis: Homes with 3000 sq.ft. or more have higher than average tax dollar count`

### Are the number of bathrooms related to the house value?
`Null Hypothesis: The number of bathrooms is independent to the value of the home`

`Alternate Hypothesis: The number of bathrooms is related to the value of the home`

### Are the number of bedrooms related to the house value?
`Null Hypothesis: The number of bedrooms is independent to the value of the home`

`Alternate Hypothesis: The number of bedrooms is related to the value of the home`

In [1]:
# imports

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import norm, ttest_ind, pearsonr
from math import sqrt

from scipy import stats

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.preprocessing import PolynomialFeatures

import env
import acquire
import prepare

## Acquire

In [2]:
# acquire data 

df = acquire.get_zillow_data()

In [3]:
# Take a look at the data

df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,11721753,616260,NaN,NaN,NaN,2.0,3.0,None,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,1018,0.017785,2017-07-21
1,11289917,2061546,1.0,NaN,NaN,2.0,3.0,None,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
2,11705026,1834372,NaN,NaN,NaN,1.0,2.0,None,6.0,1.0,...,35606.0,2016.0,23624.0,543.69,None,NaN,6.037232e+13,1772,-0.146056,2017-06-30
3,14269464,1923117,NaN,NaN,NaN,3.0,4.0,None,NaN,3.0,...,880456.0,2016.0,445569.0,9819.72,None,NaN,6.059064e+13,2028,0.021085,2017-06-01
4,11446756,451211,NaN,NaN,NaN,2.0,3.0,None,6.0,2.0,...,107110.0,2016.0,46573.0,1399.27,None,NaN,6.037650e+13,2681,-0.163064,2017-08-23


In [4]:
# Print a structured shape of the original data

print(f'My original dataframe is coming in with {df.shape[0]} rows and {df.shape[1]} columns.')

My original dataframe is coming in with 28124 rows and 62 columns.


In [5]:
# Looking at the columns with more detail

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28124 entries, 0 to 28123
Data columns (total 62 columns):
parcelid                        28124 non-null int64
id                              28124 non-null int64
airconditioningtypeid           7303 non-null float64
architecturalstyletypeid        33 non-null float64
basementsqft                    25 non-null float64
bathroomcnt                     28124 non-null float64
bedroomcnt                      28124 non-null float64
buildingclasstypeid             0 non-null object
buildingqualitytypeid           17811 non-null float64
calculatedbathnbr               28050 non-null float64
decktypeid                      213 non-null float64
finishedfloor1squarefeet        2439 non-null float64
calculatedfinishedsquarefeet    28077 non-null float64
finishedsquarefeet12            27999 non-null float64
finishedsquarefeet13            0 non-null object
finishedsquarefeet15            0 non-null object
finishedsquarefeet50            2439 non

In [6]:
# Take a look at the original max value of the features that I want adjust for outliers

df[['calculatedfinishedsquarefeet', 'taxvaluedollarcnt']].max()

calculatedfinishedsquarefeet       17245.0
taxvaluedollarcnt               23858374.0
dtype: float64

### Takeaways:
- This data set is rife with null values
- There are many redundant columns (several square footage columns, several structural columns, etc)
- There are some serious outliers that are skewing the values

## Prepare

### For prepare, I did the following:
- renamed columns to make them understandable
- dropped a total of 52 null/NAN, irrelevant, and redundant rows
- removed outliers from appraisal_value and square_feet

In [7]:
# Use the clean_zillow funtion to drop columns, drop nulls, and address outliers

df = prepare.clean_zillow(df)
df

,bathrooms,bedrooms,square_feet,fips,longitude,city,zip_code,year_built,appraisal_value,taxes
0,2.0,3.0,1316.0,6037.0,-118.355396,12447.0,95997.0,1923.0,205123.0,2627.48
1,2.0,3.0,1458.0,6037.0,-118.113100,5534.0,97318.0,1970.0,136104.0,2319.90
2,1.0,2.0,1421.0,6037.0,-118.291863,12447.0,96018.0,1911.0,35606.0,543.69
3,3.0,4.0,2541.0,6059.0,-117.912245,38032.0,96958.0,2003.0,880456.0,9819.72
4,2.0,3.0,1491.0,6037.0,-118.338253,54722.0,96162.0,1955.0,107110.0,1399.27
...,...,...,...,...,...,...,...,...,...,...
28119,2.0,6.0,2634.0,6037.0,-118.298006,12447.0,95985.0,1921.0,878190.0,12766.88
28120,1.5,4.0,1187.0,6059.0,-118.015068,16764.0,96180.0,1959.0,465999.0,5552.68
28121,3.0,5.0,3015.0,6059.0,-117.578817,13693.0,96983.0,2002.0,852813.0,11662.88
28122,3.0,5.0,2992.0,6037.0,-118.554206,12447.0,96356.0,1977.0,1120000.0,13663.03


In [8]:
# Take a look at my post-cleaning dataframe, which is much more manageable.

print(f'My cleaned dataframe is now coming in with {df.shape[0]} rows and {df.shape[1]} columns.')

My cleaned dataframe is now coming in with 24903 rows and 10 columns.


In [9]:
# Take a more detailed look at my post-cleaning dataframe. I now have 24903 rows, all columns are either float 
# or int, and no nulls

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24903 entries, 0 to 28123
Data columns (total 10 columns):
bathrooms          24903 non-null float64
bedrooms           24903 non-null float64
square_feet        24903 non-null float64
fips               24903 non-null float64
longitude          24903 non-null float64
city               24903 non-null float64
zip_code           24903 non-null float64
year_built         24903 non-null float64
appraisal_value    24903 non-null float64
taxes              24903 non-null float64
dtypes: float64(10)
memory usage: 2.1 MB


In [10]:
# Calculate the tax rate and add it as a column

df['tax_rate'] = (df['taxes'] / df['appraisal_value'])

## Deliverable: Where the Properties Are Located

In [11]:
# The FIPS code shows that the data is based on 3 counties in Southern California: Los Angeles, Orange, and Ventura 

df['fips'].unique()

array([6037., 6059., 6111.])

In [12]:
# Add the name of the county based on the FIPS numbers from the db

df['county_name'] = df['fips'].map({6037: 'Los Angeles', 6059: 'Orange', 6111: 'Ventura'})
df

,bathrooms,bedrooms,square_feet,fips,longitude,city,zip_code,year_built,appraisal_value,taxes,tax_rate,county_name
0,2.0,3.0,1316.0,6037.0,-118.355396,12447.0,95997.0,1923.0,205123.0,2627.48,0.012809,Los Angeles
1,2.0,3.0,1458.0,6037.0,-118.113100,5534.0,97318.0,1970.0,136104.0,2319.90,0.017045,Los Angeles
2,1.0,2.0,1421.0,6037.0,-118.291863,12447.0,96018.0,1911.0,35606.0,543.69,0.015270,Los Angeles
3,3.0,4.0,2541.0,6059.0,-117.912245,38032.0,96958.0,2003.0,880456.0,9819.72,0.011153,Orange
4,2.0,3.0,1491.0,6037.0,-118.338253,54722.0,96162.0,1955.0,107110.0,1399.27,0.013064,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...
28119,2.0,6.0,2634.0,6037.0,-118.298006,12447.0,95985.0,1921.0,878190.0,12766.88,0.014538,Los Angeles
28120,1.5,4.0,1187.0,6059.0,-118.015068,16764.0,96180.0,1959.0,465999.0,5552.68,0.011916,Orange
28121,3.0,5.0,3015.0,6059.0,-117.578817,13693.0,96983.0,2002.0,852813.0,11662.88,0.013676,Orange
28122,3.0,5.0,2992.0,6037.0,-118.554206,12447.0,96356.0,1977.0,1120000.0,13663.03,0.012199,Los Angeles


In [13]:
county_distribution = pd.DataFrame(df.groupby('county_name').county_name.count())
county_distribution.columns = ['count']
county_distribution

,count
county_name,
Los Angeles,15961
Orange,6710
Ventura,2232


## Deliverable: the Distribution of Tax Rates for Each County

In [ ]:
# Showing the median tax rate per county

county_median_tax_rate = pd.DataFrame(round(df.groupby('county_name').tax_rate.median(), 5))
county_median_tax_rate.columns = ['tax_rate']
county_median_tax_rate

In [ ]:
# Visualize the tax distribution by county

los_angeles_tax_dist = df[df.county_name == "Los Angeles"].tax_rate
orange_tax_dist = df[df.county_name == "Orange"].tax_rate
ventura_tax_dist = df[df.county_name == "Ventura"].tax_rate

In [ ]:
los_angeles_tax_dist.describe()

In [ ]:
# Visualization showing side-by-side comparison of County Tax Rate

plt.figure(figsize=(10,6))

plt.subplot(1,3,1)
sns.distplot(los_angeles_tax_dist, bins=100, kde=True, rug=True)
plt.xlim(0, .10)
plt.ylim(0, 300)
plt.title("Los Angeles County Tax Distribution")

plt.subplot(1,3,2)
sns.distplot(orange_tax_dist, bins=100, kde=True, rug=True, color='orange')
plt.xlim(0, .10)
plt.ylim(0, 300)
plt.title("Orange County Tax Distribution")

plt.subplot(1,3,3)
sns.distplot(ventura_tax_dist, bins=100, kde=True, rug=True, color='green')
plt.xlim(0, .10)
plt.ylim(0, 300)
plt.title("Ventura County Tax Distribution")

plt.tight_layout()

plt.show()

In [ ]:
# After reviewing the pairwise correlations, I've decided on my features 

features = [
    "bedrooms",
    "bathrooms",
    "square_feet",
    "appraisal_value"
]

df = df[features]
df.head()

In [ ]:
# Take a look at 'square_feet', 'appraisal_value' with the outliers removed

df[['square_feet', 'appraisal_value']].describe()

In [ ]:
# Construct a figure with subplots of histograms of features

# Sets size, can adjust 
plt.subplots(figsize=(25, 15))

plt.subplot(2,2,1);
line1 = plt.hist(df.square_feet)
plt.title('square feet');

plt.subplot(2,2,2);
line2 = plt.hist(df.bedrooms);
plt.title('bed_count');

plt.subplot(2,2,3);
line3 = plt.hist(df.bathrooms);
plt.title('bath_count');

plt.subplot(2,2,4);
line3 = plt.hist(df.appraisal_value);
plt.title('tax_value');

In [ ]:
# Split the data into train, validate, and test and then confirm the shape of the split sets

train, validate, test = prepare.split(df)

In [ ]:
sns.heatmap(df.corr(),cmap="Blues")

In [ ]:
sns.pairplot(df, corner=True)

In [ ]:
# Visualize the original variables versus the scaled variables

plt.figure(figsize=(15, 10))

plt.subplot(321)
plt.hist(train.square_feet, bins=25, ec='black')
plt.title('Original Square Feet')
plt.subplot(322)
plt.hist(train.square_feet_scaled, bins=25, ec='black')
plt.title('MinMax Scaled Square Feet')

plt.subplot(323)
plt.hist(train.bedrooms, bins=25, ec='black')
plt.title('Original Bed Count')
plt.subplot(324)
plt.hist(train.bedrooms_scaled, bins=25, ec='black')
plt.title('MinMax Scaled Bed Count')

plt.subplot(325)
plt.hist(train.bathrooms, bins=25, ec='black')
plt.title('Original Bath Count')
plt.subplot(326)
plt.hist(train.bathrooms_scaled, bins=25, ec='black')
plt.title('MinMax Scaled Bath Count')

### Takeaways:
- The majority of homes have 3 bedrooms and 2 bathrooms
- The average square footage of the properties is 1721 sq.ft.

## Explore

In [ ]:
def plot_variable_pairs_regline(train, hue=None):
    '''
    This function takes in a df and displays a pairplot
    with a red regression line.
    '''
    plot_kws={'line_kws':{'color':'red'}, 
              'scatter_kws': {'alpha': 0.7}}
    sns.pairplot(train.drop(['square_feet','bedrooms','bathrooms'],axis=1), hue=hue, kind="reg", plot_kws={'line_kws':{'color':'red'}, 
                                                    'scatter_kws': {'alpha': 0.1}})

In [ ]:
plot_variable_pairs_regline(train, hue=None)

### T-Test: Does the Square-footage of a home affect Tax Value?

$H_0$: The average price of homes with 3000 sq.ft. or more is equal to the average price of homes with 3000 sq.ft. or less.

$H_A$: Homes with 3000 sq.ft. or more have higher than average tax dollar count.

In [ ]:
# Perform the T-Test

df_less_than_3000 = train[train.square_feet <= 3000]
df_greater_than_3000 = train[train.square_feet > 3000]

a = df_less_than_3000.appraisal_value.mean()
b = df_greater_than_3000.appraisal_value.mean()

t, p = ttest_ind(df_greater_than_3000.appraisal_value,
                df_less_than_3000.appraisal_value)

if p > a:
    print(f'''Because p ({p:.4f}) > alpha (.05), we fail to reject the Null Hypothesis''')
else:
    print(f'''Because p ({p:.4f}) < alpha (.05), we reject the Null Hypothesis''')
print(p)

In [ ]:
# View/confirm values

df_less_than_3000.appraisal_value.mean()

In [ ]:
# View/confirm values

df_greater_than_3000.appraisal_value.mean()

### Correlation Test1: Are the number of bathrooms related to the house value?

$H_0$: The number of bathrooms is independent to the value of the home 

$H_A$: The number of bathrooms is related to the value of the home

In [ ]:
confidence_interval = 0.95
a = 1 - confidence_interval 

corr, p = stats.pearsonr(train.bathrooms, train.appraisal_value)
print(corr, p)

if p > a:
    print(f'''Because p ({p:.4f}) > alpha (.05), we fail to reject the Null Hypothesis''')
else:
    print(f'''Because p ({p:.4f}) < alpha (.05), we reject the Null Hypothesis''')

### Correlation Test2: Are the number of bedrooms related to the house value?

$H_0$: The number of bedrooms is independent to the value of the home 

$H_A$: The number of bedrooms is related to the value of the home

In [ ]:
# Perform the Correlation Test

corr, p = stats.pearsonr(train.bedrooms, train.appraisal_value)
print(corr, p)

if p > a:
    print(f'''Because p ({p:.4f}) > alpha (.05), we fail to reject the Null Hypothesis''')
else:
    print(f'''Because p ({p:.4f}) < alpha (.05), we reject the Null Hypothesis''')

### Takeaways: 
- The low p-value on all of the tests indicate a relationship between my target and independent variables


## Modeling

In [ ]:
# Assign X and y to my features and my target variable, respectively

X_train = train[['square_feet_scaled', 'bedrooms_scaled', 'bathrooms_scaled']]
X_validate = validate[['square_feet_scaled', 'bedrooms_scaled', 'bathrooms_scaled']]
X_test = test[['square_feet_scaled', 'bedrooms_scaled', 'bathrooms_scaled']]

y_train = train[['appraisal_value']]
y_validate = validate[['appraisal_value']]
y_test = test[['appraisal_value']]

In [ ]:
# Validate that my X_train contains what I think it contains

X_train.head()

In [ ]:
# Validate that my y_train contains what I think it contains

y_train.head()

In [ ]:
# Establish my baseline

# 1. Compute appraisal_value_pred_mean
appraisal_value_pred_mean = y_train.appraisal_value.mean()
y_train['appraisal_value_pred_mean'] = appraisal_value_pred_mean
y_validate['appraisal_value_pred_mean'] = appraisal_value_pred_mean

# 2. Compute appraisal_value_pred_median
appraisal_value_pred_median = y_train.appraisal_value.median()
y_train['appraisal_value_pred_median'] = appraisal_value_pred_median
y_validate['appraisal_value_pred_median'] = appraisal_value_pred_median

# 3. RMSE of appraisal_value_pred_mean
rmse_train = mean_squared_error(y_train.appraisal_value, y_train.appraisal_value_pred_mean) ** .5
rmse_validate = mean_squared_error(y_validate.appraisal_value, y_validate.appraisal_value_pred_mean) ** .5

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

# 4. RMSE of appraisal_value_pred_median
rmse_train = mean_squared_error(y_train.appraisal_value, y_train.appraisal_value_pred_median) ** .5
rmse_validate = mean_squared_error(y_validate.appraisal_value, y_validate.appraisal_value_pred_median) ** .5
print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

In [ ]:
# Following Madeleine's idea, I'm creating a dataframe for comparison of metrics for model selection

from sklearn.metrics import explained_variance_score

metric_df = pd.DataFrame(data=[{
    'model': 'mean_baseline', 
    'RMSE_validate': rmse_validate,
    'r^2_validate': explained_variance_score(y_validate.appraisal_value, y_validate.appraisal_value_pred_mean)}])

metric_df

In [ ]:
# First model: Linear Regression (OLS)

# make the thing
 
lm = LinearRegression(normalize=True)

# fit the model to our training data
# 
# fit the thing
# 
lm.fit(X_train, y_train.appraisal_value)

# predict train
# 
# use the thing!
# 
y_train['appraisal_value_pred_lm'] = lm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.appraisal_value, y_train.appraisal_value_pred_lm) ** (1/2)

# predict validate
y_validate['appraisal_value_pred_lm'] = lm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.appraisal_value, y_validate.appraisal_value_pred_lm) ** (1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# Add OLS Regressor to my metric dataframe

metric_df = metric_df.append({
    'model': 'OLS Regressor', 
    'RMSE_validate': rmse_validate,
    'r^2_validate': explained_variance_score(y_validate.appraisal_value, y_validate.appraisal_value_pred_lm)}, ignore_index=True)

metric_df

In [ ]:
# LassoLars

# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train.appraisal_value)

# predict train
y_train['appraisal_value_pred_lars'] = lars.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.appraisal_value, y_train.appraisal_value_pred_lars) ** (1/2)

# predict validate
y_validate['appraisal_value_pred_lars'] = lars.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.appraisal_value, y_validate.appraisal_value_pred_lars) ** (1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# Add LassoLars to my metric dataframe

metric_df = metric_df.append({
    'model': 'Lasso alpha 1', 
    'RMSE_validate': rmse_validate,
    'r^2_validate': explained_variance_score(y_validate.appraisal_value, y_validate.appraisal_value_pred_lars)}, ignore_index=True)

metric_df

In [ ]:
# 

# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train)

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate)
X_test_degree2 = pf.transform(X_test)

In [ ]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train, y_train.appraisal_value)

# predict train
y_train['appraisal_value_pred_lm2'] = lm2.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.appraisal_value, y_train.appraisal_value_pred_lm2) ** (1/2)

# predict validate
y_validate['appraisal_value_pred_lm2'] = lm2.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.appraisal_value, y_validate.appraisal_value_pred_lm2)

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# Add Polynomial model to my metric df

metric_df = metric_df.append({
    'model': 'Quadratic Model', 
    'RMSE_validate': rmse_validate,
    'r^2_validate': explained_variance_score(y_validate.appraisal_value, y_validate.appraisal_value_pred_lm2)}, ignore_index=True)

metric_df

### Model Takeaways:
 
- OLS Regressor and Lasso/Lars appeared to perform largely the same and both beat my baseline
- The Quadratic model appeared to perform much worse and did not come close to beating the baseline

In [ ]:
# Pick the OLS model and run my test

y_test = pd.DataFrame(y_test)

# predict on test
y_test['appraisal_value_pred_lm'] = lm.predict(X_test)

# evaluate: rmse
rmse_test = mean_squared_error(y_test.appraisal_value, y_test.appraisal_value_pred_lm) ** (1/2)

print("RMSE for OLS Model using LinearRegression\nOut-of-Sample Performance: ", rmse_test)

### Test Takeaway:
- My test data performed very comparably to my train and validate datasets. 
- The test data outperfomed my baseline.

## Conclusion

- The data suggests that there is a correlation between the square footage, the number of bathrooms and bedrooms and the home's value.

- Moving forward, keeping in mind that location is the key factor for home value, I would love to explore data that had more detailed information about nearby school disticts and local amenities.